In [4]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [5]:

options = Options()

options.add_argument('--user-data-dir=/Users/Administrator/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Default')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--start-maximized")

options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

# Add a preference to disable image loading


capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True




In [4]:
file_path = 'urls.txt'

categories = []

with open(file_path, 'r') as file:
    for line in file:     
        category = line.strip()
        categories.append(category)
file_path = 'cities.txt'

cities = []

with open(file_path, 'r') as file:
    for line in file:     
        cityy = line.strip().lower()
        cities.append(cityy)

In [32]:
all_details = {}
index = 0
urls = []
names = []
error = ""
history = 1
x = 0
for count,category in enumerate(categories):
    driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
    for city in cities:
        if history%24==0 :
            try:
                driver.close()
            except:
                error = "driver close problem in history part"
                x+=1
            time.sleep(3)
            driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
            history += 1
        else:
            history += 1
        try:
            wb_address = category + "?grid_view=1"
            wb_address = wb_address.replace("impcat",city)
            driver.get(wb_address)
            
            time.sleep(4)
            html_text = driver.page_source
            soup = BeautifulSoup(html_text,"lxml")
            
            # If captcha appears
            while "check the box below to proceed" in html_text:
                print("Stucked in the Captcha , please check the box to continue","r")
                html_text = driver.page_source
                soup = BeautifulSoup(html_text,"lxml")
                continue

            if "No results found for" in html_text:
                continue
            if "Oh no" in html_text:
                continue


            soup_old = ""
            print(f'Category Number:{count+1}/{len(categories)}----Dict_Length:{len(all_details)}',end='\r')
            while True:


                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                html_text = driver.page_source
                soup = BeautifulSoup(html_text,"lxml")
                if soup == soup_old:
                    break
                else:
                    soup_old = soup
                try:

                    btn = driver.find_element(By.CLASS_NAME,"fm2")
                    driver.execute_script("arguments[0].click();",btn)

                except:
                    error = "error in finding button"
                    x+=1
                    break
            html_text = driver.page_source
            soup = BeautifulSoup(html_text,"lxml")
            products = soup.find("div",class_="q_hm1 cnhdr fxmn")
            products = products.find("div",class_='lay-left')
            products = products.find("ul",class_="mListGrp w100 sid_df fww wlm mFrgn")
            sections = products.find_all("li")
            filtered_sections = []

            for section in sections:
                if section.has_attr('id'):
                    filtered_sections.append(section)

            for section in filtered_sections:
                details={}
                try:
                    name = section.find("h3").text
                    price = section.find("p").text
                    try:
                        url = section.find('li',class_="mListPrc").find("a")["href"]
                    except:
                        try:
                            url = section.find("a")["href"]
                        except:
                            url = None
                        
                except:
                    
                    continue

                details["name"]= name
                details["price"] = price
                details["url"] = url
                details["category"] = category.split("/")[-1].split(".html")[0]


                all_details[index] = details
                index+=1
        except:
            error = "error in big try"
            x+=1
            continue
    try:
        driver.close()
    except:
        error = "error in final driver close "
        x+=1
        
    time.sleep(2)

In [33]:
df = pd.DataFrame().from_dict(all_details).T
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.to_csv("indiamart.csv",index=False)
df

,name,price,url,category
0,Natural Hot Washed PET Bottle Flake,₹60/ Kg,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-flake
1,Unwashed Light Blue PET Flakes,₹48/ Kg,https://www.indiamart.com/proddetail/light-blu...,pet-bottle-flake
2,"White Pet Flakes, For Bottle Making, Packaging...",₹74/ Kg,https://www.indiamart.com/proddetail/pet-flake...,pet-bottle-flake
3,PET Bottles Flakes,,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-flake
4,"Natural Hot Washed pet flakes, Packaging Type:...",₹65/ Kg,https://www.indiamart.com/proddetail/pet-flake...,pet-bottle-flake
...,...,...,...,...
90713,LDPE Scrap,,https://www.indiamart.com/punarv-waste-managers/,ldpe-scrap
90714,LDPE Plastic Scrap,,None,ldpe-scrap
90715,LDPE Pet Bottles Scrap,,https://www.indiamart.com/shree-laxmi-sai-scrap/,ldpe-scrap
90716,Milky White Ldpe Scrap Rolls,₹85/ Kg,https://www.indiamart.com/proddetail/ldpe-scra...,ldpe-scrap


In [34]:
df = pd.read_csv("indiamart.csv")
df.head()

,name,price,url,category
0,Natural Hot Washed PET Bottle Flake,₹60/ Kg,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-flake
1,Unwashed Light Blue PET Flakes,₹48/ Kg,https://www.indiamart.com/proddetail/light-blu...,pet-bottle-flake
2,"White Pet Flakes, For Bottle Making, Packaging...",₹74/ Kg,https://www.indiamart.com/proddetail/pet-flake...,pet-bottle-flake
3,PET Bottles Flakes,NaN,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-flake
4,"Natural Hot Washed pet flakes, Packaging Type:...",₹65/ Kg,https://www.indiamart.com/proddetail/pet-flake...,pet-bottle-flake


In [35]:
df["price"] = df["price"].apply(lambda x: str(x).strip(" "))
df["price"] = df["price"].apply(lambda x: str(x).lower())

df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kg","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("KG","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram(s)","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("KILOGRAMS","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kgs","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kilogram","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kg(s)","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("per kg","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("kgs","kg"))

df["price"] = df["price"].apply(lambda x: str(x).replace("kg onwards","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Kilogram Onwards","kg"))
df["price"] = df["price"].apply(lambda x: str(x).replace("TONNE","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("Tonne","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("metric ton(s)","tonne"))
df["price"] = df["price"].apply(lambda x: str(x).replace("ton","tonne") if str(x).split("/")[-1]==" ton" else x)
df["price"] = df["price"].apply(lambda x: str(x).replace("metric ton","tonne"))

df["price"] = df["price"].apply(lambda x: str(x).replace("Ton","tonne"))



In [36]:
df["metric"] = df["price"].apply(lambda x: str(x).split("/")[-1].strip())

{'nan': 68446,
 'kg': 12595,
 'tonne': 2077,
 'piece': 283,
 'लोहे की रद्दी': 262,
 'iron scrap': 252,
 'कॉपर स्क्रैप': 150,
 'metric': 149,
 'पीतल का रद्दी माल': 123,
 'scrap': 109,
 'application:  metal industry': 105,
 'copper scrap': 86,
 'अँल्युमीनीअम की रद्दी': 85,
 'metal scrap': 70,
 'aluminium scrap': 66,
 'स्टेनलेस स्टील स्क्रैप': 65,
 'country of origin:  made in india': 63,
 'वेस्ट पेपर': 54,
 'कॉपर तार का स्क्रैप': 52,
 'cast iron scrap': 50,
 'color:  black': 50,
 'ferrous scrap': 49,
 'heavy metal scraps': 47,
 'कंप्यूटर का रद्दी माल': 46,
 'brass scrap': 46,
 'ms scrap': 44,
 'material:  aluminium': 43,
 'packaging size:  100 kg': 42,
 'copper wire scrap': 42,
 'जस्ता कीट': 41,
 'लेड स्क्रैप': 40,
 'tin scrap': 39,
 'waste paper': 36,
 'packaging type:  loose': 35,
 'shape:  wire': 34,
 'steel scrap': 33,
 'stainless steel scrap': 33,
 'heavy metal scrap': 32,
 'packaging size:  50 kg': 32,
 'corrugated box scrap': 31,
 'electronic scrap': 31,
 'color:  blue': 31,
 'ind

In [42]:
df[df["metric"]=="pet content:  90- 95%"]["url"][152]

'https://www.indiamart.com/proddetail/pet-scrap-20769283012.html'

In [37]:
names_to_filter = ["kg", "tonne", "metric"]
filtered_df = df[df['metric'].isin(names_to_filter)]
filtered_df = filtered_df.reset_index(drop=True)

In [38]:
filtered_df.to_csv("filtered_indiamart.csv",index=False)

In [24]:
filtered_df

,name,price,url,category,metric
0,Natural Hot Washed PET Bottle Flake,₹60/ kg,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-flake,kg
1,Unwashed Light Blue PET Flakes,₹48/ kg,https://www.indiamart.com/proddetail/light-blu...,pet-bottle-flake,kg
2,"White Pet Flakes, For Bottle Making, Packaging...",₹74/ kg,https://www.indiamart.com/proddetail/pet-flake...,pet-bottle-flake,kg
3,"Natural Hot Washed pet flakes, Packaging Type:...",₹65/ kg,https://www.indiamart.com/proddetail/pet-flake...,pet-bottle-flake,kg
4,Mixed Pet Bottles Flakes,₹18/ kg,https://www.indiamart.com/proddetail/pet-bottl...,pet-bottle-flake,kg
...,...,...,...,...,...
12590,"Ab 2 Grade YELLOW Aluminium Bronze Scrap Ab2, ...",₹600/ kg,https://www.indiamart.com/proddetail/aluminium...,aluminum-bronze-scrap,kg
12591,"Aluminium Bronze Scrap, 50 Kg",₹450/ kg,https://www.indiamart.com/proddetail/aluminium...,aluminum-bronze-scrap,kg
12592,Aluminum Bronze Scrap,₹450/ kg,https://www.indiamart.com/proddetail/aluminum-...,aluminum-bronze-scrap,kg
12593,"Non Ferrous YELLOW Aluminium Bronze Scrap, For...",₹500/ kg,https://www.indiamart.com/proddetail/aluminium...,aluminum-bronze-scrap,kg
